# Module 00: Introduction and Stock Returns

**Difficulty**: ⭐ (Beginner)

**Estimated Time**: 30-40 minutes

**Prerequisites**: 
- Basic arithmetic (addition, subtraction, multiplication, division)
- Understanding of percentages (what does 5% mean?)
- No prior stock market knowledge required!

## Learning Objectives

By the end of this notebook, you will be able to:
1. Calculate **absolute returns** and **percentage returns** for any stock
2. Explain **why percentages are essential** in stock analysis (vs absolute values)
3. Compute **daily, weekly, and monthly returns** from real Malaysian stock data
4. Understand **compound returns** and how they differ from simple returns
5. Apply these concepts to analyze Maybank (1155.KL) and Top Glove (5225.KL)

## Why This Matters

**Stock returns are the foundation of ALL technical indicators.**

Every indicator you'll learn (RSI, MACD, Bollinger Bands, etc.) ultimately measures some aspect of returns:
- **Moving averages** → average price over time (returns from a baseline)
- **RSI** → relative strength of positive vs negative returns
- **MACD** → momentum and convergence of returns
- **Bollinger Bands** → volatility of returns

Understanding returns mathematically is your first step to understanding WHY indicators work!

---

## Setup

First, let's import the libraries we need and configure our environment.

In [ ]:
# Data manipulation and numerical operations
import pandas as pd
import numpy as np

# Data acquisition - downloading Malaysian stock data
import yfinance as yf

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Display more rows and columns in pandas DataFrames
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 20)
pd.set_option('display.width', 1000)

# Set random seed for reproducibility (if we use any random operations later)
np.random.seed(42)

print("✓ All libraries imported successfully!")
print("✓ Environment configured")
print("\nYou're ready to start learning!")

---

## Part 1: What is a Stock Return?

### The Simple Question: Did I Make Money?

When you buy a stock and sell it later, you want to know: **"Did I make money?"**

There are TWO ways to answer this:

#### 1. Absolute Return (in Ringgit)
**Formula**: `Selling Price - Buying Price`

**Example**: 
- You buy Maybank at RM 8.50
- You sell at RM 9.00
- Absolute return = RM 9.00 - RM 8.50 = **RM 0.50 profit**

#### 2. Percentage Return (relative to investment)
**Formula**: `(Selling Price - Buying Price) / Buying Price × 100%`

**Same Example**:
- Percentage return = (9.00 - 8.50) / 8.50 × 100% = **5.88% profit**

### Why Do We Need BOTH?

Let's see why percentage returns are crucial...

In [ ]:
# Example: Comparing two stocks
# Stock A: Maybank (lower price)
stock_a_buy = 8.50
stock_a_sell = 9.00

# Stock B: Top Glove (higher price during pandemic)
stock_b_buy = 25.00
stock_b_sell = 25.50

# Calculate absolute returns
absolute_return_a = stock_a_sell - stock_a_buy
absolute_return_b = stock_b_sell - stock_b_buy

# Calculate percentage returns
percentage_return_a = (absolute_return_a / stock_a_buy) * 100
percentage_return_b = (absolute_return_b / stock_b_buy) * 100

# Display results
print("=" * 60)
print("COMPARING TWO STOCKS: WHICH PERFORMED BETTER?")
print("=" * 60)
print(f"\nStock A (Maybank):")
print(f"  Buy Price:  RM {stock_a_buy:.2f}")
print(f"  Sell Price: RM {stock_a_sell:.2f}")
print(f"  Absolute Return: RM {absolute_return_a:.2f}")
print(f"  Percentage Return: {percentage_return_a:.2f}%")

print(f"\nStock B (Top Glove):")
print(f"  Buy Price:  RM {stock_b_buy:.2f}")
print(f"  Sell Price: RM {stock_b_sell:.2f}")
print(f"  Absolute Return: RM {absolute_return_b:.2f}")
print(f"  Percentage Return: {percentage_return_b:.2f}%")

print("\n" + "=" * 60)
print("ANALYSIS:")
print("=" * 60)
print(f"Both stocks gave RM 0.50 absolute return.")
print(f"BUT Stock A gave {percentage_return_a:.2f}% while Stock B gave {percentage_return_b:.2f}%")
print(f"\n→ Stock A performed BETTER because you got MORE return per Ringgit invested!")
print(f"\nThis is why we use PERCENTAGE returns in technical analysis.")

### Key Insight: Percentages Allow Fair Comparison

**Why percentage returns matter:**
1. **Fair comparison** between different stocks (regardless of price)
2. **Standardized measurement** - 5% return means the same thing for all stocks
3. **Scalable** - works whether you invest RM 1,000 or RM 100,000
4. **Universal** - all technical indicators use percentage-based calculations

**From now on, when we say "return" we mean percentage return (unless specified otherwise).**

---

## Part 2: Real Malaysian Stock Data

Let's work with actual data from Bursa Malaysia! We'll download historical prices for:
- **Maybank (1155.KL)** - Malaysia's largest bank
- **Top Glove (5225.KL)** - World's largest glove manufacturer

### Downloading Data with yfinance

In [ ]:
# Download 1 year of data for Maybank
print("Downloading Maybank (1155.KL) data...")
maybank_data = yf.download('1155.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Download 1 year of data for Top Glove
print("Downloading Top Glove (5225.KL) data...")
topglove_data = yf.download('5225.KL', start='2023-01-01', end='2024-01-01', progress=False)

# Validate data was downloaded successfully
assert len(maybank_data) > 0, "Failed to download Maybank data. Check internet connection."
assert len(topglove_data) > 0, "Failed to download Top Glove data. Check internet connection."

print(f"\n✓ Downloaded {len(maybank_data)} days of Maybank data")
print(f"✓ Downloaded {len(topglove_data)} days of Top Glove data")
print("\nData downloaded successfully!")

In [ ]:
# Let's look at the first few rows of Maybank data
print("First 5 days of Maybank trading data:")
print(maybank_data.head())

### Understanding the Columns

Each row is one trading day, with these columns:
- **Open**: Price when market opened (9:00 AM)
- **High**: Highest price during the day
- **Low**: Lowest price during the day
- **Close**: Price when market closed (5:00 PM) ← **We'll use this for returns**
- **Adj Close**: Adjusted close (accounts for dividends, splits)
- **Volume**: Number of shares traded

**For calculating returns, we use the "Close" price** because it represents the final agreed price of the day.

---

## Part 3: Calculating Daily Returns

**Daily return** = How much the stock changed from yesterday's close to today's close.

### The Formula

$$
\text{Daily Return (\%)} = \frac{\text{Today's Close} - \text{Yesterday's Close}}{\text{Yesterday's Close}} \times 100
$$

Let's calculate this for Maybank!

In [ ]:
# Extract just the closing prices for Maybank
maybank_close = maybank_data['Close']

# Calculate daily returns using the formula
# pct_change() does exactly this calculation for us!
maybank_daily_returns = maybank_close.pct_change() * 100  # Multiply by 100 to get percentage

# The first row will be NaN (no previous day to compare to)
print("First 10 days of Maybank daily returns:")
print(maybank_daily_returns.head(10))

print("\nNote: Day 1 is NaN because there's no 'yesterday' to compare to.")

In [ ]:
# Let's verify the calculation manually for one day
# Take day 2 and day 3 as an example
day2_close = maybank_close.iloc[1]
day3_close = maybank_close.iloc[2]

manual_return = ((day3_close - day2_close) / day2_close) * 100
pct_change_return = maybank_daily_returns.iloc[2]

print("Manual calculation verification:")
print(f"Day 2 close: RM {day2_close:.4f}")
print(f"Day 3 close: RM {day3_close:.4f}")
print(f"\nManual calculation: {manual_return:.4f}%")
print(f"pct_change() result: {pct_change_return:.4f}%")
print(f"\nDifference: {abs(manual_return - pct_change_return):.10f}% (should be ~0)")
print("\n✓ Our formula matches pandas pct_change()!")

### Visualizing Daily Returns

Let's see how Maybank's daily returns look over the year:

In [ ]:
# Create a visualization of daily returns
plt.figure(figsize=(14, 6))

# Plot daily returns as a line
plt.subplot(1, 2, 1)
plt.plot(maybank_daily_returns.index, maybank_daily_returns.values, linewidth=1, alpha=0.7)
plt.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
plt.title('Maybank Daily Returns (2023)', fontsize=14, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Daily Return (%)')
plt.grid(True, alpha=0.3)

# Plot as histogram to see distribution
plt.subplot(1, 2, 2)
plt.hist(maybank_daily_returns.dropna(), bins=50, edgecolor='black', alpha=0.7)
plt.axvline(x=0, color='red', linestyle='--', linewidth=2)
plt.title('Distribution of Daily Returns', fontsize=14, fontweight='bold')
plt.xlabel('Daily Return (%)')
plt.ylabel('Frequency (Number of Days)')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Left chart: Daily returns over time (red line = 0% = no change)")
print("Right chart: How often each return level occurred (histogram)")

In [ ]:
# Calculate some statistics about the daily returns
print("=" * 60)
print("MAYBANK DAILY RETURNS STATISTICS (2023)")
print("=" * 60)
print(f"\nMean (Average) daily return: {maybank_daily_returns.mean():.4f}%")
print(f"Median daily return: {maybank_daily_returns.median():.4f}%")
print(f"\nBest day (highest return): {maybank_daily_returns.max():.2f}%")
print(f"Worst day (lowest return): {maybank_daily_returns.min():.2f}%")
print(f"\nStandard deviation (volatility): {maybank_daily_returns.std():.4f}%")
print(f"\nPositive return days: {(maybank_daily_returns > 0).sum()} days")
print(f"Negative return days: {(maybank_daily_returns < 0).sum()} days")
print(f"Unchanged days: {(maybank_daily_returns == 0).sum()} days")

### What Do These Numbers Tell Us?

1. **Mean return**: The average daily movement (positive = stock trending up)
2. **Standard deviation**: How much returns vary (high = volatile, risky)
3. **Best/Worst day**: Extreme movements (useful for risk assessment)
4. **Positive vs Negative days**: Overall sentiment and trend

These statistics are the foundation for many technical indicators!

---

## Part 4: Compound Returns (Multi-Day Returns)

### The Problem with Adding Returns

**Question**: If a stock returns +10% on Monday and +10% on Tuesday, is the total return 20%?

**Answer**: NO! It's actually 21%.

Here's why...

In [ ]:
# Example: Compounding effect
initial_price = 100.00

# Day 1: +10% return
day1_return = 0.10  # 10% as decimal
price_after_day1 = initial_price * (1 + day1_return)

# Day 2: another +10% return (but on the NEW price)
day2_return = 0.10  # 10% as decimal
price_after_day2 = price_after_day1 * (1 + day2_return)

# Calculate total return
total_return = ((price_after_day2 - initial_price) / initial_price) * 100

print("=" * 60)
print("COMPOUND RETURNS EXAMPLE")
print("=" * 60)
print(f"\nStarting price: RM {initial_price:.2f}")
print(f"\nDay 1: +10% return")
print(f"  Price after Day 1: RM {price_after_day1:.2f}")
print(f"\nDay 2: +10% return")
print(f"  10% of RM {price_after_day1:.2f} = RM {price_after_day1 * 0.10:.2f}")
print(f"  Price after Day 2: RM {price_after_day2:.2f}")
print(f"\nTotal return: {total_return:.2f}%")
print(f"\nIf we just added: 10% + 10% = 20% (WRONG!)")
print(f"Actual compound return: {total_return:.2f}% (CORRECT!)")
print(f"\nDifference: {total_return - 20:.2f}% (extra profit from compounding)")

### The Compound Return Formula

To calculate return over multiple days:

$$
\text{Total Return} = \left(\prod_{i=1}^{n} (1 + r_i)\right) - 1
$$

Where:
- $r_i$ = daily return as decimal (5% = 0.05)
- $\prod$ = multiply all values together
- $n$ = number of days

**In plain English**: Multiply all (1 + daily_return) together, then subtract 1.

Let's calculate Maybank's total return for 2023:

In [ ]:
# Calculate total return for Maybank in 2023 (compound method)
# Convert percentage returns to decimal (divide by 100)
maybank_returns_decimal = maybank_daily_returns / 100

# Add 1 to each return
maybank_growth_factors = 1 + maybank_returns_decimal

# Multiply all together (use .prod() to multiply all values)
total_growth = maybank_growth_factors.prod()

# Subtract 1 and convert to percentage
total_return_compound = (total_growth - 1) * 100

# Compare to wrong method (just summing)
total_return_wrong = maybank_daily_returns.sum()

# Also calculate using actual prices (direct method)
first_price = maybank_close.iloc[0]
last_price = maybank_close.iloc[-1]
total_return_direct = ((last_price - first_price) / first_price) * 100

print("=" * 60)
print("MAYBANK TOTAL RETURN FOR 2023")
print("=" * 60)
print(f"\nFirst trading day price: RM {first_price:.4f}")
print(f"Last trading day price:  RM {last_price:.4f}")
print(f"\n1. Direct calculation (last/first - 1):")
print(f"   Total return = {total_return_direct:.2f}%")
print(f"\n2. Compound method (multiply daily returns):")
print(f"   Total return = {total_return_compound:.2f}%")
print(f"\n3. Wrong method (sum daily returns):")
print(f"   Total return = {total_return_wrong:.2f}% (INCORRECT!)")
print(f"\nDifference: {abs(total_return_direct - total_return_compound):.6f}%")
print("(Direct and compound methods should match!)")

---

## Part 5: Exercises

Now it's your turn to practice! Try to solve these before looking at the solutions.

### Exercise 1: Calculate Top Glove Returns

Calculate the daily returns for Top Glove (5225.KL) using the data we downloaded earlier.

**Tasks**:
1. Extract the closing prices from `topglove_data`
2. Calculate daily percentage returns
3. Print the first 10 daily returns
4. Calculate and print the mean, max, and min returns

In [ ]:
# Your code here




<details>
<summary><b>Click here for solution</b></summary>

```python
# Extract closing prices
topglove_close = topglove_data['Close']

# Calculate daily returns
topglove_daily_returns = topglove_close.pct_change() * 100

# Print first 10 returns
print("First 10 days of Top Glove daily returns:")
print(topglove_daily_returns.head(10))

# Calculate statistics
print(f"\nMean return: {topglove_daily_returns.mean():.4f}%")
print(f"Max return: {topglove_daily_returns.max():.2f}%")
print(f"Min return: {topglove_daily_returns.min():.2f}%")
```
</details>

### Exercise 2: Compare Volatility

Which stock is more volatile (risky) - Maybank or Top Glove?

**Hint**: Volatility is measured by standard deviation of returns. Higher standard deviation = more volatile.

**Tasks**:
1. Calculate standard deviation for both stocks' daily returns
2. Compare and print which one is more volatile
3. Calculate the ratio (how many times more volatile is one vs the other)

In [ ]:
# Your code here




<details>
<summary><b>Click here for solution</b></summary>

```python
# Calculate standard deviations
maybank_volatility = maybank_daily_returns.std()
topglove_volatility = topglove_daily_returns.std()

print("Volatility Comparison:")
print(f"Maybank volatility: {maybank_volatility:.4f}%")
print(f"Top Glove volatility: {topglove_volatility:.4f}%")

if topglove_volatility > maybank_volatility:
    ratio = topglove_volatility / maybank_volatility
    print(f"\nTop Glove is MORE volatile")
    print(f"It's {ratio:.2f}x more volatile than Maybank")
else:
    ratio = maybank_volatility / topglove_volatility
    print(f"\nMaybank is MORE volatile")
    print(f"It's {ratio:.2f}x more volatile than Top Glove")
```
</details>

### Exercise 3: Monthly Returns

Instead of daily returns, calculate **monthly returns** for Maybank.

**Hint**: Use `resample('M')` to group data by month, then use `.last()` to get the last price of each month.

**Tasks**:
1. Resample Maybank closing prices to monthly frequency
2. Calculate monthly percentage returns
3. Print all monthly returns for 2023
4. Which month had the best return? Which had the worst?

In [ ]:
# Your code here




<details>
<summary><b>Click here for solution</b></summary>

```python
# Resample to monthly, taking last price of each month
maybank_monthly = maybank_close.resample('M').last()

# Calculate monthly returns
maybank_monthly_returns = maybank_monthly.pct_change() * 100

print("Maybank Monthly Returns (2023):")
print(maybank_monthly_returns)

# Find best and worst months
best_month = maybank_monthly_returns.idxmax()
worst_month = maybank_monthly_returns.idxmin()

print(f"\nBest month: {best_month.strftime('%B %Y')} with {maybank_monthly_returns.max():.2f}%")
print(f"Worst month: {worst_month.strftime('%B %Y')} with {maybank_monthly_returns.min():.2f}%")
```
</details>

### Exercise 4: Compound Return Verification

Verify that the compound return formula works correctly.

**Tasks**:
1. Take the first 5 days of Maybank returns
2. Calculate the 5-day total return using the compound formula
3. Verify by comparing with direct calculation (day 5 price / day 1 price - 1)
4. Show that simple addition gives the wrong answer

In [ ]:
# Your code here




<details>
<summary><b>Click here for solution</b></summary>

```python
# Get first 5 days of returns (skip NaN)
first_5_returns = maybank_daily_returns.iloc[1:6]  # Skip first NaN

# Method 1: Compound formula
compound = ((1 + first_5_returns / 100).prod() - 1) * 100

# Method 2: Direct calculation
price_day1 = maybank_close.iloc[0]
price_day5 = maybank_close.iloc[5]
direct = ((price_day5 - price_day1) / price_day1) * 100

# Method 3: Simple sum (WRONG)
simple_sum = first_5_returns.sum()

print("5-Day Return Calculation:")
print(f"\nCompound formula: {compound:.4f}%")
print(f"Direct calculation: {direct:.4f}%")
print(f"Simple sum (wrong): {simple_sum:.4f}%")
print(f"\nDifference (compound vs direct): {abs(compound - direct):.8f}%")
print(f"Error from simple sum: {abs(simple_sum - compound):.4f}%")
```
</details>

---

## Summary

Congratulations! You've completed Module 00. Let's review what you learned:

### Key Concepts Mastered:

1. **Absolute vs Percentage Returns**
   - Absolute: Ringgit gained/lost
   - Percentage: Return relative to investment (used in technical analysis)

2. **Daily Return Calculation**
   - Formula: `(Today - Yesterday) / Yesterday × 100`
   - Used `pct_change()` in pandas

3. **Return Statistics**
   - Mean: Average return
   - Standard deviation: Volatility/risk
   - Max/Min: Extreme movements

4. **Compound Returns**
   - Can't simply add returns
   - Must multiply growth factors: $(1 + r_1) \times (1 + r_2) \times ... - 1$

5. **Real Malaysian Stock Data**
   - Used yfinance to download data
   - Analyzed Maybank (1155.KL) and Top Glove (5225.KL)

### Why This Matters for Technical Indicators:

Every indicator you'll learn builds on these concepts:
- **Moving Averages** (Module 04): Average of returns over time
- **RSI** (Module 05): Ratio of positive to negative returns
- **Bollinger Bands** (Module 06): Standard deviation of returns
- **MACD** (Module 07): Momentum of returns

### What's Next?

In **Module 01: Averages and Central Tendency**, you'll learn:
- Mean, median, mode in depth
- Weighted averages (foundation for EMA)
- How averaging reduces noise in stock data
- Introduction to Simple Moving Average (SMA)

### Additional Practice

Before moving to Module 01, try:
1. Download data for other Malaysian stocks (CIMB: 1023.KL, Axiata: 6888.KL)
2. Calculate weekly and monthly returns
3. Compare volatility across different sectors
4. Identify which stocks had positive mean returns in 2023

---

## Additional Resources

### Further Reading:
- [Investopedia: Rate of Return](https://www.investopedia.com/terms/r/rateofreturn.asp)
- [Investopedia: Compound Annual Growth Rate](https://www.investopedia.com/terms/c/cagr.asp)
- [Bursa Malaysia](https://www.bursamalaysia.com) - Official exchange

### Python Documentation:
- [Pandas pct_change()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.pct_change.html)
- [yfinance documentation](https://pypi.org/project/yfinance/)

---

**Great work!** When you're ready, move on to Module 01 to continue your mathematical journey in stock trading.